# Q3

In [2]:
import requests
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

Get the BTC and ETH data from binance

In [4]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_binance_data(symbol, interval, start_time, end_time):
    base_url = "https://api.binance.com/api/v3/klines"
    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': start_time,
        'endTime': end_time,
        'limit': 1000 
    }
    
    data = []
    while True:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        klines = response.json()
        if not klines:
            break
        data.extend(klines)
        params['startTime'] = klines[-1][0] + 1
        if len(klines) < 1000:
            break
    
    df = pd.DataFrame(data, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume', 
        'close_time', 'quote_asset_volume', 'number_of_trades', 
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df.astype(float)
    
    return df

# Define start and end times
end_time = int(datetime.now().timestamp() * 1000)
start_time = int((datetime.now() - timedelta(days=30)).timestamp() * 1000)

# Fetch BTC and ETH minute data
btc_data = get_binance_data('BTCUSDT', '1m', start_time, end_time)
eth_data = get_binance_data('ETHUSDT', '1m', start_time, end_time)